In [2]:
import pandas as pd
import datetime
import numpy as np

In [3]:
wines_ = pd.read_excel("data/Subset1.xls", 1)
print(min(wines_['DOY']))

213.0


On ne peut donc au maximum prendre comme data que les 213 jours précédent la récolte, une information supplémentaire n'est probablement pas importante car on va surtout analyser l'effet de la météo d'une année sur le vin

In [4]:
wines_.dropna(subset = ['Harvest date'], inplace=True)
wines_.reset_index(drop=True, inplace=True)

In [5]:
weather = pd.read_excel("data/Subset3.xls", [1,2,3,4,5])
weather[1]

,Year,Month,day,Tave,Tmax,Tmin,DOY,Unnamed: 7,Year.1,GST,WI,HI
0,1911,1,1,8.35,16.7,0.0,1,NaN,1911.0,15.073364,1099.050000,967.254052
1,1911,1,2,8.85,19.4,-1.7,2,NaN,1912.0,15.752336,1246.550000,1146.450272
2,1911,1,3,6.65,16.1,-2.8,3,NaN,1913.0,16.722430,1446.300000,1276.779404
3,1911,1,4,8.30,18.3,-1.7,4,NaN,1914.0,15.678579,1217.966001,1103.276910
4,1911,1,5,7.80,18.9,-3.3,5,NaN,1915.0,15.950196,1277.291993,1133.617078
...,...,...,...,...,...,...,...,...,...,...,...,...
36885,2011,12,27,6.95,15.6,-1.7,361,NaN,NaN,NaN,NaN,NaN
36886,2011,12,28,11.65,19.4,3.9,362,NaN,NaN,NaN,NaN,NaN
36887,2011,12,29,11.15,16.7,5.6,363,NaN,NaN,NaN,NaN,NaN
36888,2011,12,30,12.20,15.0,9.4,364,NaN,NaN,NaN,NaN,NaN


In [6]:
i = list(range(1,201))
i = list(map(str, i))
iavg = ['avg' + item for item in i]
imin = ['min' + item for item in i]
imax = ['max' + item for item in i]
temp_avg = pd.DataFrame(columns = iavg)
temp_min = pd.DataFrame(columns = imin)
temp_max = pd.DataFrame(columns = imax)

In [7]:
wines = wines_.copy()

In [8]:
for i in range(len(wines)):
    new_row = []
    r = wines["Reg"][i]
    d = wines['Harvest date'][i]
    h = int(wines['DOY'][i])
    new_row = list(weather[r].loc[(weather[r]['Year'] == d.year) & (weather[r]['DOY'].isin(range(h-200,h)))]['Tave'])
    if len(new_row) != 200:
        new_row = [float('nan')] * 200
    temp_avg.loc[len(temp_avg)] = new_row

In [9]:
for i in range(len(wines)):
    new_row = []
    r = wines["Reg"][i]
    d = wines['Harvest date'][i]
    h = int(wines['DOY'][i])
    new_row = list(weather[r].loc[(weather[r]['Year'] == d.year) & (weather[r]['DOY'].isin(range(h-200,h)))]['Tmin'])
    if len(new_row) != 200:
        new_row = [float('nan')] * 200
    temp_min.loc[len(temp_min)] = new_row

In [10]:
for i in range(len(wines)):
    new_row = []
    r = wines["Reg"][i]
    d = wines['Harvest date'][i]
    h = int(wines['DOY'][i])
    new_row = list(weather[r].loc[(weather[r]['Year'] == d.year) & (weather[r]['DOY'].isin(range(h-200,h)))]['Tmax'])
    if len(new_row) != 200:
        new_row = [float('nan')] * 200
    temp_max.loc[len(temp_max)] = new_row


In [11]:
print(temp_avg.isnull().sum(axis = 0))
print(temp_min.isnull().sum(axis = 0))
print(temp_max.isnull().sum(axis = 0))

avg1      102
avg2      102
avg3      102
avg4      102
avg5      102
         ... 
avg196    102
avg197    102
avg198    102
avg199    102
avg200    102
Length: 200, dtype: int64
min1      102
min2      102
min3      102
min4      102
min5      102
         ... 
min196    102
min197    102
min198    102
min199    102
min200    102
Length: 200, dtype: int64
max1      102
max2      102
max3      102
max4      102
max5      102
         ... 
max196    102
max197    102
max198    102
max199    102
max200    102
Length: 200, dtype: int64


we have the same missing values for Tmin, Tmax and Taverage

In [12]:
temperatures = temp_avg.copy()

In [13]:
mapping = {'White': -1, 'Red': 1}

# Apply the mapping to the 'category' column using the map() function
wines['Color'] = wines['Color'].map(mapping)

wines

,Table.No.,Cultivar,Color,Recommend,RecReg,Reg,Harvest date,DOY,Must.Brix,Must.Tacid,Must.pH,Wine.Alcohol,Wine.Facid,Wine.Extract,Wine.Tannin,Wine.Tasting
0,9,Chardony,-1,Particularly recommended,1.0,3,1936-09-08,251.0,27.2,0.54,3.88,15.6,0.42,4.0,0.03,"Very distinct; good, but alcoholic"
1,9,Chardony,-1,Particularly recommended,1.0,4,1936-08-31,243.0,23.5,0.67,3.79,13.0,0.39,NaN,NaN,Distinct; lacks acid; standard quality
2,9,Chardony,-1,Particularly recommended,1.0,1,1937-10-01,273.0,22.1,0.72,3.38,12.0,0.52,2.0,0.03,Light body; fruity; distinct; good quality
3,9,Chardony,-1,Particularly recommended,1.0,3,1937-09-11,253.0,24.6,0.63,3.69,12.8,0.49,2.1,0.03,Flat; tends to oxidize; but distinct
4,9,Chardony,-1,Particularly recommended,1.0,1,1938-09-14,256.0,19.8,0.78,3.19,10.5,0.68,2.8,0.07,"Tart, but palatable; distinct; picked too early"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,25,Red Malaga (Molinera),1,Table grape,NaN,4,1937-10-25,297.0,21.8,0.47,3.74,12.0,0.28,2.3,0.04,Very flat; some spoilage; very ordiry sherry
1469,25,Red Malaga (Molinera),1,Table grape,NaN,4,1940-10-07,280.0,22.7,0.36,3.90,12.4,0.40,2.8,0.04,Flat; pink; very common; lacks everything
1470,25,Ribier,1,Table grape,NaN,4,1935-11-01,304.0,20.2,0.55,NaN,10.8,0.34,2.5,0.08,"Flat; poor; sherry; soft, but no character"
1471,25,Ribier,1,Table grape,NaN,4,1937-10-13,285.0,17.6,0.50,3.58,9.5,0.35,2.1,0.06,Flat; ordiry; tends to spoil


In [14]:
dataset = pd.concat([wines,temp_avg], axis = 1)

In [15]:
print(dataset.isna().sum()) #il faudrait trouver et enlever les 102 wines ou on a pas la temperature selon mon avis 

Table.No.       0
Cultivar        0
Color           0
Recommend       0
RecReg       1025
             ... 
avg196        102
avg197        102
avg198        102
avg199        102
avg200        102
Length: 216, dtype: int64


# Mise en place d'une analyse avec tout d'abord la color du wine comme attribute et le ph à prédire

In [16]:
copie = dataset.copy()
#enlever les colonnes qu'on ne compte pas utiliser
copie = copie.drop(columns = ['Table.No.', 'Cultivar', 'Recommend', 'RecReg', 'Reg', 'Wine.Alcohol',
       'Harvest date', 'DOY', 'Must.Brix', 'Must.Tacid', 'Wine.Facid', 'Wine.Extract', 'Wine.Tannin',
       'Wine.Tasting'])
#verifier qu'on a nos colonnes désirées
copie.columns 

Index(['Color', 'Must.pH', 'avg1', 'avg2', 'avg3', 'avg4', 'avg5', 'avg6',
       'avg7', 'avg8',
       ...
       'avg191', 'avg192', 'avg193', 'avg194', 'avg195', 'avg196', 'avg197',
       'avg198', 'avg199', 'avg200'],
      dtype='object', length=202)

In [17]:
columns = list(temperatures.columns)
columns.append('Color')
columns.append('Must.pH')
#delete row with value = Nan
#voir si pour les y on peut pas remplacer par une valeur :
# d'autres possibilités (ffill et bfill qui propagent les valeurs observées aux voisines Nan)
# imputation par la moyenne (possible aussi avec la mediane)
# exemple :           movies_df['budget'].fillna(movies_df['budget'].mean(), inplace=True)
copie.dropna(subset = columns, inplace=True)
copie.reset_index(drop=True, inplace=True)

In [18]:
# selection of class to predict : here ph
y = copie[['Must.pH']]

# temperature and color extraction
colonnes = list(temperatures.columns)
colonnes.append('Color')

# récupération des valeurs de toutes les colonnes à la suite
X = copie[colonnes]

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), iavg)  
    ],
    remainder='passthrough' 
)


In [23]:
import numpy as np
from sklearn.linear_model import LassoCV, RidgeCV

#linear regression avec lasso et ridge 
alphas = np.logspace(-5, 2, num=100)
alphas2 = 1/alphas

pLassoCv = Pipeline([
    ('preprocessor', preprocessor),
    ('reg', LassoCV(alphas=alphas, cv=5))
])

pRidgeCv = Pipeline([
    ('preprocessor', preprocessor),
    ('reg', RidgeCV(alphas=alphas2, cv=5))
])

pLassoCv.fit(X_train, y_train.values.ravel())
pRidgeCv.fit(X_train, y_train.values.ravel())
print("Lasso Best alpha:", pLassoCv['reg'].alpha_)
print("Ridge Best alpha:", 1/pRidgeCv['reg'].alpha_)



Lasso Best alpha: 0.0029836472402833374
Ridge Best alpha: 0.004862601580065354


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

In [ ]:
c_values = np.logspace(-3, .2, num=3)


In [ ]:
param_grid_lin = {'C': c_values, 
              'epsilon': [0.01, 0.1, 1]}
grid_lin = GridSearchCV(SVR(kernel='linear'), param_grid=param_grid_lin, cv=5)

grid_lin.fit(X_train, y_train.values.ravel())

print("Best hyperparameters: ", grid_lin.best_params_)
print("Cross-validation score: ", grid_lin.best_score_)

Best hyperparameters:  {'C': 0.001, 'epsilon': 0.1}
Cross-validation score:  0.2517434390994759


In [ ]:
param_grid_rbf = {'C': c_values, 
              'epsilon': [0.01, 0.1, 1],
              'gamma': ['scale', 'auto']}
grid_rbf = GridSearchCV(SVR(kernel='rbf'), param_grid=param_grid_rbf, cv=5)

grid_rbf.fit(X_train, y_train.values.ravel())

print("Best hyperparameters: ", grid_rbf.best_params_)
print("Cross-validation score: ", grid_rbf.best_score_)

Best hyperparameters:  {'C': 1.5848931924611136, 'epsilon': 0.1, 'gamma': 'scale'}
Cross-validation score:  0.39902530301112876


In [ ]:
param_grid_poly = {'C': c_values, 
              'epsilon': [0.01, 0.1, 1],
              'degree':[2,3,4,5,6,7]}
grid_poly = GridSearchCV(SVR(kernel='poly'), param_grid=param_grid_poly, cv=5)

grid_poly.fit(X_train, y_train.values.ravel())

print("Best hyperparameters: ", grid_poly.best_params_)
print("Cross-validation score: ", grid_poly.best_score_)
print("Cross-validation results: ", grid_poly.cv_results_)
print(grid_poly.scorer_)

Best hyperparameters:  {'C': 1.5848931924611136, 'degree': 2, 'epsilon': 0.1}
Cross-validation score:  0.378283937890696
Cross-validation results:  {'mean_fit_time': array([0.05428967, 0.04080009, 0.00495009, 0.0501955 , 0.03991108,
       0.0046032 , 0.05437837, 0.0385972 , 0.00461278, 0.05219455,
       0.04129391, 0.00487256, 0.05240431, 0.0397337 , 0.00386305,
       0.05367384, 0.04076686, 0.00457907, 0.05199809, 0.03550863,
       0.00393257, 0.05183883, 0.03695207, 0.00396543, 0.05036168,
       0.03877077, 0.00412431, 0.05353174, 0.03890567, 0.00382619,
       0.05315509, 0.04298539, 0.00451851, 0.0556232 , 0.04081607,
       0.00396957, 0.0710896 , 0.04625721, 0.00394764, 0.06379151,
       0.04924078, 0.00449996, 0.06314988, 0.04424906, 0.00443153,
       0.06435165, 0.04381313, 0.00474405, 0.06830764, 0.04722004,
       0.00412068, 0.06464543, 0.04564519, 0.0048316 ]), 'std_fit_time': array([0.00091838, 0.0024764 , 0.00091556, 0.00094251, 0.00108804,
       0.00025584, 0.002

In [ ]:
param_grid_sig = {'C': c_values, 
              'epsilon': [0.01, 0.1, 1]}
grid_sig = GridSearchCV(SVR(kernel='sigmoid'), param_grid=param_grid_sig, cv=5)

grid_sig.fit(X_train, y_train.values.ravel())

print("Best hyperparameters: ", grid_sig.best_params_)
print("Cross-validation score: ", grid_sig.best_score_)

Best hyperparameters:  {'C': 0.039810717055349734, 'epsilon': 0.1}
Cross-validation score:  0.12050367751341962


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np


# Define the hyperparameter grid for GridSearchCV
param_grid_rt = {
    'n_estimators': [10, 50, 100],
    'max_depth': [2, 4, 6, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Create a KFold cross-validation object
kf = KFold(n_splits=5, shuffle=True, random_state=123)

# Create a Random Forest model
rf = RandomForestRegressor()

# Create a GridSearchCV object
grid_rt = GridSearchCV(estimator=rf, param_grid=param_grid_rt, cv=5, scoring='neg_mean_squared_error', n_jobs=2)

# Fit the GridSearchCV object to the data
grid_rt.fit(X_train, y_train.values.ravel())

# Print the best hyperparameters and the corresponding mean cross-validation score
print("Best hyperparameters:", grid_rt.best_params_)
print("Best cross-validation score:", -grid_rt.best_score_)


Best hyperparameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}
Best cross-validation score: 0.04495346454188211


In [ ]:
from scipy.stats import ttest_rel
from scipy.stats import shapiro


model1 = grid_poly
model2 = grid_lin

# Extract the cross-validation scores for each model
scores1 = model1.cv_results_['mean_test_score']
scores2 = model2.cv_results_['mean_test_score']

# Check for normality using the Shapiro-Wilk test
_, p1 = shapiro(scores1)
_, p2 = shapiro(scores2)

# Perform a paired t-test if the scores are normally distributed
if p1 > 0.05 and p2 > 0.05:
    _, p = ttest_rel(scores1, scores2)
    print('Paired t-test p-value:', p)
else:
    print('The scores are not normally distributed')

The scores are not normally distributed


In [ ]:
len(scores1)

54

In [ ]:
from scipy.stats import wilcoxon

# Perform a Wilcoxon signed-rank test if the scores are not normally distributed
w, p = wilcoxon(scores1, scores2)
print("Wilcoxon signed-rank test:")
print("Statistic:", w)
print("p-value:", p)

ValueError: The samples x and y must have the same length.